In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "/mnt/cephfs/echoi/codes/l1/checkpoints/deepscaler/l1_exact/actor/global_step_2000"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype="auto",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

/root/anaconda3/envs/sft/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.79s/it]


In [2]:
import os
os.environ['WANDB_MODE'] = 'offline'

In [3]:
from datasets import load_dataset

NUM_CALIBRATION_SAMPLES = 256
MAX_SEQUENCE_LENGTH = 4096

# Load and preprocess the dataset
ds = load_dataset("garage-bAInd/Open-Platypus", split="train")
# ds = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")
ds = ds.shuffle(seed=42).select(range(NUM_CALIBRATION_SAMPLES))

def preprocess(example):
    # return {"text": tokenizer.apply_chat_template(example["instruction"], tokenize=False)}
    return {"text": tokenizer.apply_chat_template(example["output"], tokenize=False)}
ds = ds.map(preprocess)

def tokenize(sample):
    return tokenizer(sample["text"], padding=False, max_length=MAX_SEQUENCE_LENGTH, truncation=True, add_special_tokens=False)
ds = ds.map(tokenize, remove_columns=ds.column_names)

Map: 100%|██████████| 256/256 [00:00<00:00, 3733.94 examples/s]


In [4]:
from compressed_tensors.quantization import (
    QuantizationArgs,
    QuantizationScheme,
    QuantizationStrategy,
    QuantizationType,
) 
from llmcompressor.transformers import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier
recipe = GPTQModifier(
    targets="Linear",
    config_groups={
        "config_group": QuantizationScheme(
            targets=["Linear"],
            weights=QuantizationArgs(
                num_bits=4,
                type=QuantizationType.INT,
                strategy=QuantizationStrategy.GROUP,
                group_size=128,
                symmetric=False,
                dynamic=True,
            ),
        ),
    },
    ignore=["lm_head"],
    dampening_frac=0.01
)

ValidationError: 1 validation error for QuantizationArgs
  Value error, ('One of token or tensor must be used for dynamic ', 'quantization') [type=value_error, input_value={'num_bits': 4, 'type': <... False, 'dynamic': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [ ]:
# Apply quantization

oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

# Save the compressed model
SAVE_DIR = "./l1-2000steps" + "-W4A16-G128-open-platyplus-clip"
model.save_pretrained(SAVE_DIR, save_compressed=True)
tokenizer.save_pretrained(SAVE_DIR)

2025-03-31T14:25:23.538232+0000 | main | WARNING - Process rank: 0, device: cuda:0, n_gpu: 8, distributed training: True, 16-bits training: False
2025-03-31T14:25:23.541807+0000 | main | INFO - Training/evaluation parameters TrainingArguments(
_n_gpu=8,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


2025-03-31T14:25:30.424778+0000 | pre_initialize_structure | INFO - Compression lifecycle structure pre-initialized for 0 modifiers


2025-03-31T14:25:35.971139+0000 | pre_initialize_structure | INFO - Compression lifecycle structure pre-initialized for 0 modifiers


2025-03-31T14:25:41.019550+0000 | one_shot | INFO - *** One Shot ***


...

2025-03-31T14:30:23.293349+0000 | finalize | INFO - Compression lifecycle finalized for 1 modifiers


Checking whether model follows 2:4 sparsity structure: 100%|██████████| 197/197 [00:00<00:00, 1144.85it/s]

2025-03-31T14:30:23.497292+0000 | get_model_compressor | INFO - Inferring a sparsity configuration requires a global sparsity calculation. This can be costly for large models. To skip the calculation of compression statistics set skip_compression_stats=True



Calculating model sparsity: 100%|██████████| 731/731 [00:05<00:00, 125.07it/s]
Checking whether model follows 2:4 sparsity structure: 100%|██████████| 197/197 [00:00<00:00, 3104.46it/s]
Calculating quantization compression ratio: 312it [00:00, 588.49it/s]
Quantized Compression: 100%|██████████| 731/731 [00:07<00:00, 100.02it/s]


('./l1-2000steps-W4A16-G128-open-platyplus/tokenizer_config.json',
 './l1-2000steps-W4A16-G128-open-platyplus/special_tokens_map.json',
 './l1-2000steps-W4A16-G128-open-platyplus/tokenizer.json')